In [17]:
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

In [18]:
import os
from os import listdir

In [19]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [20]:
from tensorflow.keras.preprocessing import image_dataset_from_directory, image

train_directory = r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
valid_directory = r'../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

train_dataset = image_dataset_from_directory(train_directory,
                                            seed = 2059,
                                            image_size = (160, 160),
                                            batch_size = 16)


valid_dataset = image_dataset_from_directory(valid_directory,
                                            seed = 2059,
                                            image_size = (160, 160),
                                            batch_size = 16)

Found 10000 files belonging to 2 classes.
Found 800 files belonging to 2 classes.


In [21]:
labels = train_dataset.class_names
print(len(labels))

2


In [22]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3, 3), input_shape = (160, 160, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))

model.add(Dropout(0.2))

model.add(Dense(len(labels), activation = 'softmax'))

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# ModelCheckpoint to save model in case of 
# interrupting the learning process
checkpoint = ModelCheckpoint("./face_mask_classifierCC.h5",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor = 'val_loss',
                          restore_best_weights = True,
                          patience = 3,  # number of epochs with 
                                       # no improvement after which 
                                       # training will be stopped
                          verbose=1)

callbacks = [earlystop, checkpoint]
model.compile(loss = 'sparse_categorical_crossentropy',
                        optimizer = Adam(learning_rate = 0.01),
                        metrics = ['accuracy'])

In [24]:
epochs = 20

history = model.fit(
    train_dataset,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = valid_dataset)

model.save("./face_mask_classifierCC.h5")

Epoch 1/20
625/625 [==============================] - 18s 28ms/step - loss: 0.4173 - accuracy: 0.8909 - val_loss: 0.1991 - val_accuracy: 0.9375

Epoch 00001: val_loss improved from inf to 0.19906, saving model to ./face_mask_classifierCC.h5
Epoch 2/20
625/625 [==============================] - 18s 28ms/step - loss: 0.1928 - accuracy: 0.9344 - val_loss: 2.5276 - val_accuracy: 0.7188

Epoch 00002: val_loss did not improve from 0.19906
Epoch 3/20
625/625 [==============================] - 17s 26ms/step - loss: 0.1305 - accuracy: 0.9609 - val_loss: 0.1371 - val_accuracy: 0.9525

Epoch 00003: val_loss improved from 0.19906 to 0.13709, saving model to ./face_mask_classifierCC.h5
Epoch 4/20
625/625 [==============================] - 17s 27ms/step - loss: 0.1066 - accuracy: 0.9639 - val_loss: 0.0411 - val_accuracy: 0.9850

Epoch 00004: val_loss improved from 0.13709 to 0.04109, saving model to ./face_mask_classifierCC.h5
Epoch 5/20
625/625 [==============================] - 17s 27ms/step - los